In [1]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
from scipy import sparse 
import itertools

import matplotlib.pyplot as plt
import collections
from sklearn.cluster import KMeans
from sklearn import metrics

from sklearn.decomposition import PCA
from umap import UMAP

from py_pcha import PCHA

from matplotlib.colors import LinearSegmentedColormap

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu



In [2]:
# outdir     = "/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results"
outdir_fig = "/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/figures"

In [3]:
from scipy.io import mmread

f1 = "../../data/fly_t4t5/GSM3592260_T4T5_48_matrix.mtx.gz"
mat = mmread(f1)
mat

<17492x3894 sparse matrix of type '<class 'numpy.int64'>'
	with 6341865 stored elements in COOrdinate format>

In [4]:
f2 = "../../data/fly_t4t5/GSM3592260_T4T5_48_genes.tsv.gz"
gene = pd.read_csv(f2, sep='\t', header=None)
gene

,0,1
0,FBgn0031081,Nep3
1,FBgn0052826,tRNA:Pro-CGG-1-1
2,FBgn0031085,CG9570
3,FBgn0062565,Or19b
4,FBgn0031088,CG15322
...,...,...
17487,FBgn0040725,CG13946
17488,FBgn0031277,CG13947
17489,FBgn0041250,Gr21a
17490,FBgn0031279,CG3544


In [5]:
f3 = "../../data/fly_t4t5/GSM3592260_T4T5_48_barcodes.tsv.gz"
cell = pd.read_csv(f3, sep='\t', header=None)
cell

,0
0,AAACCTGAGAATAGGG-1
1,AAACCTGAGCCACTAT-1
2,AAACCTGAGCTTTGGT-1
3,AAACCTGAGGGCATGT-1
4,AAACCTGCACCGTTGG-1
...,...
3889,TTTGTCAGTAAAGGAG-1
3890,TTTGTCAGTCGAACAG-1
3891,TTTGTCAGTTTGTTGG-1
3892,TTTGTCATCAACGGGA-1


In [6]:
f4 = "../../data/fly_t4t5/GSM3592260_clust48.txt.gz"
types = pd.read_csv(f4, sep='\t')
types

,barcode,cluster
0,AAACCTGAGAATAGGG,T4a
1,AAACCTGAGCCACTAT,T4d
2,AAACCTGAGCTTTGGT,T4d
3,AAACCTGAGGGCATGT,T4d
4,AAACCTGCACCGTTGG,T5c
...,...,...
3889,TTTGTCAGTAAAGGAG,T4b
3890,TTTGTCAGTCGAACAG,T4c
3891,TTTGTCAGTTTGTTGG,T4b
3892,TTTGTCATCAACGGGA,T4b


In [7]:
meta = pd.merge(cell, types, left_index=True, right_index=True).sort_index()
meta

,0,barcode,cluster
0,AAACCTGAGAATAGGG-1,AAACCTGAGAATAGGG,T4a
1,AAACCTGAGCCACTAT-1,AAACCTGAGCCACTAT,T4d
2,AAACCTGAGCTTTGGT-1,AAACCTGAGCTTTGGT,T4d
3,AAACCTGAGGGCATGT-1,AAACCTGAGGGCATGT,T4d
4,AAACCTGCACCGTTGG-1,AAACCTGCACCGTTGG,T5c
...,...,...,...
3889,TTTGTCAGTAAAGGAG-1,TTTGTCAGTAAAGGAG,T4b
3890,TTTGTCAGTCGAACAG-1,TTTGTCAGTCGAACAG,T4c
3891,TTTGTCAGTTTGTTGG-1,TTTGTCAGTTTGTTGG,T4b
3892,TTTGTCATCAACGGGA-1,TTTGTCATCAACGGGA,T4b


In [8]:
np.all(meta['barcode'] == meta[0].apply(lambda x: x.split('-')[0]))

True

In [9]:
adata = anndata.AnnData(X=mat.T.tocsr(), 
                        obs=meta[['barcode', 'cluster']],#.astype(str), 
                        var=pd.DataFrame(index=gene[1].values.astype(str)),
                       ) #.set_index(1))
adata

/u/home/f/f7xiesnm/.conda/envs/napari/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 3894 × 17492
    obs: 'barcode', 'cluster'

In [10]:
fout = "../../data/fly_t4t5/GSM3592260_T4T5_48.h5ad"
adata.write(fout)